In [490]:
# Importing libraries
import pandas as pd
from os import listdir
from os.path import isfile, join
import xgboost as xgb
from datetime import datetime

# Reading the data
data_path = 'Data/'
log = pd.read_csv(data_path + 'LOG_WEB_201708.csv', sep=';')
clients = pd.read_csv(data_path + 'clients.csv', sep = ';')
cmd = pd.read_csv(data_path + 'CMD.csv', sep = ';')
cmd = cmd[cmd['ORDER_DATE'] >= '2017-08-01']
table = pd.read_csv(data_path + 'TABLE_CONVERSION.csv', sep = ';')

# Remove unamed column
log = log.loc[:, ~log.columns.str.contains('^Unnamed')]
clients = clients.loc[:, ~clients.columns.str.contains('^Unnamed')]
cmd = cmd.loc[:, ~cmd.columns.str.contains('^Unnamed')]
table = table.loc[:, ~table.columns.str.contains('^Unnamed')]

# Adjust data types - wrong types were causing issues
clients.CLIENT_NUMBER = [str(clients) for clients in clients.CLIENT_NUMBER]
cmd.CLIENT_NUMBER = [str(cmd) for cmd in cmd.CLIENT_NUMBER]
table.CLIENT_NUMBER = [str(table) for table in table.CLIENT_NUMBER]
table.VISITOR_ID = [str(table) for table in table.VISITOR_ID]
log.VISITOR_ID = [str(log) for log in log.VISITOR_ID]

# Cannot distinguish between multiple orders on the same day, so the table has been simplified.
cmd = cmd.groupby(['CLIENT_NUMBER', 'ORDER_DATE', 'CVIC']).sum()
# Convert back to typical DataFrame type
cmd = cmd.reset_index()

In [586]:
from multiprocessing import Pool

# Join cmd + table + log

In [506]:
cmd.CLIENT_NUMBER = [str(cmd) for cmd in cmd.CLIENT_NUMBER]
log['EVENT_SHORT'] = log.EVENT_DATE.str.slice(0,10)
data = log[log.VISITOR_ID != '0']
data = pd.merge(data, table, left_on='VISITOR_ID', right_on='VISITOR_ID', how='left')
data = pd.merge(data, cmd, left_on=['CLIENT_NUMBER', 'EVENT_SHORT'], right_on=['CLIENT_NUMBER', 'ORDER_DATE'], how='left')

In [621]:
data

,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS,EVENT_SHORT,CLIENT_NUMBER,ORDER_DATE,CVIC,ORDER_NUMBER,PRE_TAX_AMOUNT
0,381225,OUI,1,2017-08-08 00:02:06,2017-08-08 00:02:06,accueil,Liens Sponsorisés,Smartphone,NaN,1,2017-08-08,2912426551072989703,NaN,NaN,NaN,NaN
1,563053,OUI,13,2017-08-08 00:01:32,2017-08-08 00:03:27,accueil,Accès Direct,Ordinateur,NaN,1,2017-08-08,-5859629033836166974,2017-08-08,False,20279565.0,280.34
2,563053,OUI,13,2017-08-08 00:01:32,2017-08-08 00:22:04,accueil,Accès Direct,Ordinateur,NaN,1,2017-08-08,-5859629033836166974,2017-08-08,False,20279565.0,280.34
3,706588,OUI,23,2017-08-08 00:01:29,2017-08-08 00:01:32,accueil,Accès Direct,Ordinateur,NaN,1,2017-08-08,-7174350147193854740,2017-08-08,False,10139782.0,19.08
4,572702,OUI,64,2017-08-08 00:03:19,2017-08-08 00:03:19,accueil,Moteurs,Tablette,101 Oxygen,1,2017-08-08,866042539231455965,2017-08-08,True,10139780.0,97.50
5,572702,OUI,64,2017-08-08 00:03:19,2017-08-08 00:09:42,accueil,Moteurs,Tablette,101 Oxygen,1,2017-08-08,866042539231455965,2017-08-08,True,10139780.0,97.50
6,572702,OUI,64,2017-08-08 00:03:19,2017-08-08 00:10:36,accueil,Moteurs,Tablette,101 Oxygen,1,2017-08-08,866042539231455965,2017-08-08,True,10139780.0,97.50
7,698042,OUI,201,2017-08-08 00:14:41,2017-08-08 00:14:41,accueil,Liens Sponsorisés,Smartphone,sm-j510fn,1,2017-08-08,3565018203965887144,NaN,NaN,NaN,NaN
8,489527,OUI,203,2017-08-08 00:15:00,2017-08-08 00:15:00,accueil,Liens Sponsorisés,Ordinateur,NaN,1,2017-08-08,5954669164526364513,NaN,NaN,NaN,NaN
9,489527,OUI,203,2017-08-08 00:15:00,2017-08-08 00:15:40,accueil,Liens Sponsorisés,Ordinateur,NaN,1,2017-08-08,5954669164526364513,NaN,NaN,NaN,NaN


In [622]:
data.to_csv('data.csv', sep = ',')

# Visitor Journey

Looking at the logs of **August 2017** <br>
**Visitor 261647** first connects at **11:25 AM**
<br> <br>
Connected through **Sponsored Links**, on a **Computer** <br>
**19 minutes:** Address entered for delivery.  <br>
**22 minutes:** Accessed via phone to view account <br>
**28 minutes:** Puchase complete.<br>
**28 minutes:** Disconnected from website, doesn't reconnect this month
<br> <br>
**187.50 Euro Spent**

# What do I want to classify?
If a customer was convinced to purchase through magazine or online.

# Useful Features
- PAGES: Are they browsing the wares or going straight to the purchase?<br>
- GLOBAL_SOURCES: What caused them to connect to the website? <br>
- DEVICE_TYPE: Maybe magazine users use different devices <br>
- CVIC - Did they use a magazine coupon <br>
- PRE_TAX_AMOUNT <br>

# Feature Engineering

- time connected <br>
- number of unique pages visited
- T/F: Placed order 
- Time of Day

# Generating the Data Matrix
- VISITOR_ID <br>
- CLIENT_NUMBER <br>
- EVENT_SHORT <br>
- GLOBAL_SOURCES <br>
- DEVICE TYPE <br>
- CVIC <br>
- PRE_TAX_AMOUNT <br>


In [520]:
tmp = data[['VISITOR_ID', 'CLIENT_NUMBER', 'EVENT_SHORT', 'ID_SESSION', 'GLOBAL_SOURCES', 'DEVICE_TYPE', 'CVIC', 'PRE_TAX_AMOUNT', 'EVENT_DATE', 'PAGES']].copy()
tmp = tmp.fillna(0)
tmp['PURCHASE'] = True
tmp.loc[tmp.PRE_TAX_AMOUNT == 0, 'PURCHASE'] = False
tmp.columns = ['VISITOR_ID', 'CLIENT_NUMBER', 'DATE', 'ID_SESSION', 'SOURCE', 'DEVICE', 'PROMO', 'EXPENSE', 'EVENT_DATE', 'PAGES', 'PURCHASE']
tmp = tmp[['VISITOR_ID', 'CLIENT_NUMBER', 'DATE', 'ID_SESSION', 'SOURCE', 'DEVICE', 'PROMO', 'EXPENSE', 'PURCHASE', 'EVENT_DATE', 'PAGES']]
tmp['MONTH'] = tmp.EVENT_DATE.str.slice(0,7)
tmp = tmp[tmp.PURCHASE == True]

In [521]:
tmp.head()

,VISITOR_ID,CLIENT_NUMBER,DATE,ID_SESSION,SOURCE,DEVICE,PROMO,EXPENSE,PURCHASE,EVENT_DATE,PAGES,MONTH
1,563053,-5859629033836166974,2017-08-08,13,Accès Direct,Ordinateur,False,280.34,True,2017-08-08 00:03:27,accueil,2017-08
2,563053,-5859629033836166974,2017-08-08,13,Accès Direct,Ordinateur,False,280.34,True,2017-08-08 00:22:04,accueil,2017-08
3,706588,-7174350147193854740,2017-08-08,23,Accès Direct,Ordinateur,False,19.08,True,2017-08-08 00:01:32,accueil,2017-08
4,572702,866042539231455965,2017-08-08,64,Moteurs,Tablette,True,97.50,True,2017-08-08 00:03:19,accueil,2017-08
5,572702,866042539231455965,2017-08-08,64,Moteurs,Tablette,True,97.50,True,2017-08-08 00:09:42,accueil,2017-08


In [583]:
X = tmp.groupby(['CLIENT_NUMBER','ID_SESSION', 'MONTH', 'PROMO']).sum().reset_index()
X['SURF_TIME'] = 0
X.head()

,CLIENT_NUMBER,ID_SESSION,MONTH,PROMO,EXPENSE,PURCHASE,SURF_TIME
0,-1008458086851481944,13945,2017-08,False,1813.00,14.0,0
1,-1008458086851481944,18235,2017-08,False,388.50,3.0,0
2,-1010661696429978923,37668,2017-08,False,403.75,19.0,0
3,-1011583376281571893,12025,2017-08,False,632.00,16.0,0
4,-1012644870962927739,30918,2017-08,True,215.01,3.0,0


In [585]:
for client in X.CLIENT_NUMBER:
    for session in set(tmp[tmp.CLIENT_NUMBER == client].ID_SESSION):
        datetime.strptime(tmp[(tmp.ID_SESSION == session) & (tmp.CLIENT_NUMBER == client)].EVENT_DATE
, '%Y-%m-%d %H:%M:%S')
        b = tmp[(tmp.ID_SESSION == session) & (tmp.CLIENT_NUMBER == client)].iloc[-1].EVENT_DATE
        a = 
        b = datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
        X[(X.CLIENT_NUMBER == client) & (X.ID_SESSION == session)].SURF_TIME = (b-a).total_seconds()/60

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


KeyboardInterrupt: 

In [619]:
def return_surftime(cn):
    session_surftime = []
    for session in set(tmp[tmp.CLIENT_NUMBER == cn].ID_SESSION):
        a = tmp[(tmp.ID_SESSION == session) & (tmp.CLIENT_NUMBER == cn)].iloc[0].EVENT_DATE
        b = tmp[(tmp.ID_SESSION == session) & (tmp.CLIENT_NUMBER == cn)].iloc[-1].EVENT_DATE
        a = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
        b = datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
        session_surftime.append((b-a).total_seconds()/60)
    return(session_surftime)

In [620]:
p = Pool(2, maxtasksperchild=1)
surftime_res = p.map(return_surftime, X.CLIENT_NUMBER)
p.close()

ValueError: time data 'nan' does not match format '%Y-%m-%d %H:%M:%S'

In [596]:
surftime_res = [item for sublist in surftime_res for item in sublist]

In [618]:
a = str(tmp[(tmp.ID_SESSION == session) & (tmp.CLIENT_NUMBER == client)].EVENT_DATE.min())
a

'2017-08-18 23:42:40'

0:00:26


'2017-08-18 23:43:06'

In [564]:
datetime.strptime('2017-08-28 12:49:24', '%Y-%m-%d %H:%M:%S')

datetime.datetime(2017, 8, 28, 12, 49, 24)

# Pickle

In [487]:
with open(data_path+'client_by_date.pickle', 'rb') as handle:
   client_by_date = pickle.load(handle)

In [504]:
client_by_date

{'2017-08': [9089815182412087301,
  -4212914366395875314,
  4412715268887150606,
  7289012290777350169,
  870352525203243039,
  -4625196522819911645,
  -2031379451723022295,
  6441927626979868712,
  -5536008905043673038,
  1510620840604106801,
  6453316122314276914,
  7278064021490204723,
  7122309643670454325,
  5897545423665627191,
  -2506306593447968698,
  5397119728221356100,
  -7771995626234806199,
  -1597876238788263863,
  6241698716701524041,
  4210234185733210197,
  -4800809858005008293,
  -7885007154358452127,
  -3132438419034341280,
  5285474030847819876,
  8590983719385694308,
  6458856923129086055,
  -1507215425971453845,
  4559583669933178992,
  -4149535339675533,
  4659956769925496947,
  -3791399531358027651,
  6905247991116759170,
  811473898246832263,
  8976259379107758213,
  -2085207140827496307,
  9173330209602764939,
  1472052141509476498,
  4482980579790848145,
  -5541232508310421348,
  3464180607936069788,
  8189145422591557790,
  7877685402709917859,
  90650985451

# Testing

In [473]:
visit = pd.read_csv(data_path + 'VISITS_DATA_15h.csv')
visit = visit.loc[:, ~visit.columns.str.contains('^Unnamed')]
#visit = visit.fillna(0)
visit = visit[data.NB_ORDERS != 0]

visit.CLIENT_NUMBER = [str(data) for data in data.CLIENT_NUMBER]
visit.VISITOR_ID = [str(data) for data in data.VISITOR_ID]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [476]:
data.columns

Index(['VISITOR_ID', 'CLIENT_NUMBER', 'CREATION_DATE', 'NB_ORDERS',
       'TOT_AMOUNT_SPENT', 'FIRST_ORDER_DATE', 'LAST_ORDER_DATE',
       'ORDER_CHANNEL', 'CVIC', 'AVG_ORDER_AMOUNT', 'ACCOUNT_SENIORITY',
       'ORDER_SENIORITY', 'ORDER_LAST_ACTIVITY', 'ORDER_FREQUENCY',
       'TOTAL_VISITS', 'LAST_VISIT', 'FIRST_VISIT',
       'AVG_PAGES_VISITED_PER_VISIT', 'VISIT_SENIORITY', 'VISIT_LAST_ACTIVITY',
       'VISIT_FREQUENCY'],
      dtype='object')

In [482]:
#data.CLIENT_NUMBER.value_counts()
data[(data.CLIENT_NUMBER == '7721669007667066057')]

,VISITOR_ID,CLIENT_NUMBER,CREATION_DATE,NB_ORDERS,TOT_AMOUNT_SPENT,FIRST_ORDER_DATE,LAST_ORDER_DATE,ORDER_CHANNEL,CVIC,AVG_ORDER_AMOUNT,...,ORDER_SENIORITY,ORDER_LAST_ACTIVITY,ORDER_FREQUENCY,TOTAL_VISITS,LAST_VISIT,FIRST_VISIT,AVG_PAGES_VISITED_PER_VISIT,VISIT_SENIORITY,VISIT_LAST_ACTIVITY,VISIT_FREQUENCY
288416,469020.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295059,476211.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296442,477686.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297777,479093.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298296,479642.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299020,480417.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304579,486251.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316586,499574.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316602,499590.0,7721669007667066057,2015-03-11,1.0,367.08,2015-05-05,2015-05-05,Internet,False,367.08,...,1233.0,1233.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# VISITOR JOURNEY

In [230]:
v_17 = log[log.VISITOR_ID == '175752']
v_17.head()

,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS
159645,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:27:43,accueil,Email marketing,Ordinateur,NaN,1
159646,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:28:14,accueil,Email marketing,Ordinateur,NaN,1
159647,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:41:45,accueil,Email marketing,Ordinateur,NaN,1
182418,175752,OUI,14817,2017-08-27 10:17:25,2017-08-27 10:17:25,accueil,Accès Direct,Ordinateur,NaN,1
184863,175752,OUI,33484,2017-08-27 14:46:12,2017-08-27 14:46:12,accueil,Accès Direct,Ordinateur,NaN,1


In [235]:
v_17.sort_values(by=['EVENT_DATE'])

,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS
159645,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:27:43,accueil,Email marketing,Ordinateur,NaN,1
2091135,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:27:51,ete-indien-2017,Email marketing,Ordinateur,NaN,1
159646,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:28:14,accueil,Email marketing,Ordinateur,NaN,1
278822,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:28:18,nouvelle-collection-automne-hiver-2017,Email marketing,Ordinateur,NaN,1
334433,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:28:30,category 21::fiche produit::6037865839848578407,Email marketing,Ordinateur,NaN,1
667488,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:31:09,ajout au panier,Email marketing,Ordinateur,NaN,1
445710,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:31:30,mon panier,Email marketing,Ordinateur,NaN,1
445711,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:31:55,mon panier,Email marketing,Ordinateur,NaN,1
607990,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:32:10,mes informations,Email marketing,Ordinateur,NaN,1
547217,175752,OUI,28914,2017-08-24 13:27:43,2017-08-24 13:32:37,ma livraison,Email marketing,Ordinateur,NaN,1
